In [50]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pandas import DataFrame

def to_int(x):
    return int(x.replace(",",""))

In [51]:
url="https://jf8q26wwud-dsn.algolia.net/1/indexes/*/queries?x-algolia-agent=Algolia%20for%20JavaScript%20(3.35.1)%3B%20Browser&x-algolia-application-id=JF8Q26WWUD&x-algolia-api-key=2bc790c0d4f44db9ab6267a597d17f1a"

payload={"requests":[{"indexName":"vnw_job_v2","params":"query=&query=&facetFilters=%5B%5D&numericFilters=%5B%5D&page=0&hitsPerPage=50&restrictSearchableAttributes=%5B%22jobTitle%22%2C%22skills%22%2C%22company%22%5D&attributesToRetrieve=%5B%22*%22%2C%22-jobRequirement%22%2C%22-jobDescription%22%5D"}]}

In [52]:
import json
r = requests.post(url, data=json.dumps(payload))

In [53]:
total_job = r.json()["results"][0]["nbHits"]

In [54]:
industry_url="https://www.vietnamworks.com/tim-viec-lam"

page = requests.get(industry_url)

soup = BeautifulSoup(page.text, 'html.parser')

In [55]:
industries = soup.find("div", class_="highlight").find_all("dd")

full_ind = []
full_no_of_job = []
for industry in industries:
  tmp = industry.get_text().split()
  no_of_jobs=str(tmp[-1])[1:-1]
  tmp = tmp[0:-1]
  ind = " ".join(tmp)
  full_ind.append(ind)
  full_no_of_job.append(int(no_of_jobs))


In [56]:
Vietnamwork_ind = {'Industry': full_ind,
                   'No. of Jobs': full_no_of_job}
Vietnamwork_result_ind = pd.DataFrame(Vietnamwork_ind, columns=['Industry', 'No. of Jobs'])
Vietnamwork_result_ind = Vietnamwork_result_ind.sort_values(by=['No. of Jobs'], ascending=False)

Vietnamwork_result_ind

Industry  No. of Jobs
11                  Tài chính/Đầu tư         1733
22                      Điện/Điện tử         1143
0                           Xây dựng         1002
23                            Cơ khí          997
10                         Kiểm toán          896
9                          Ngân hàng          880
6     Quảng cáo/Khuyến mãi/Đối ngoại          754
44              Vận chuyển/Giao nhận          709
7              Internet/Online Media          654
1        Kiến trúc/Thiết kế nội thất          575
41                  Giáo dục/Đào tạo          555
16                     Bán lẻ/Bán sỉ          513
5       Mỹ Thuật/Nghệ Thuật/Thiết Kế          506
2                       Bất động sản          465
43                            Tư vấn          462
30                  Tự động hóa/Ô tô          429
28                   Dệt may/Da giày          402
14                    Hàng tiêu dùng          398
17               Thực phẩm & Đồ uống          369
21                Nhà hàng/Khách sạn          323
32              Sản phẩm công nghiệp          311
35            Y tế/Chăm sóc sức khỏe          306
42            Y tế/Chăm sóc sức khỏe          306
45                           Kho vận          297
24                  Hóa học/Hóa sinh          282
4   Truyền hình/Truyền thông/Báo chí          276
13                          Bảo hiểm          227
29      Dược Phẩm/Công nghệ sinh học          219
20                Hàng không/Du lịch          212
3                         Viễn Thông          201
26              Điện lạnh/Nhiệt lạnh          160
18                        Thời trang          156
25        Môi trường/Xử lý chất thải          145
12                       Chứng khoán          144
33                     Công nghệ cao          128
39                   Trình dược viên          121
46                          Hàng hải          118
31            Nông nghiệp/Lâm nghiệp          113
38                           Dược sĩ           93
8                    In ấn/ Xuất bản           84
40       Phi chính phủ/Phi lợi nhuận           79
15                     Hàng gia dụng           68
27                           Dầu khí           53
19                      Hàng cao cấp           32
36                            Bác sĩ           28
34               Địa chất/Khoáng sản           25
37                        Y sĩ/Hộ lý           23

In [57]:
Vietnamwork = {'Total Jobs': total_job}

df = DataFrame(Vietnamwork, columns= ['Total Jobs'], index=[0])

In [58]:
industries = soup.find_all("div", class_="highlight")[2].find_all("dd")


full_ind = []
full_no_of_job = []

for industry in industries:
  tmp = industry.get_text().split()
  no_of_jobs=str(tmp[-1])[1:-1]
  tmp = tmp[0:-1]
  ind = " ".join(tmp)
  full_ind.append(ind)
  full_no_of_job.append(int(no_of_jobs))

demo = {'Senority': full_ind,
        'Jobs by Senority': full_no_of_job
        }

senority_pd_r = DataFrame(demo, columns= ['Senority', 'Jobs by Senority'])

In [59]:
s = senority_pd_r.set_index('Senority')
senority_pd = s.transpose()

senority_dict = senority_pd.to_dict()
# d = senority_dict.get(0)
# d
# Vietnamwork.update(senority_dict)

senority_final = {}
for key in senority_dict.keys():
    senority_final[key] = senority_dict[key]["Jobs by Senority"]
Vietnamwork.update(senority_final)

df = DataFrame(Vietnamwork, columns=['Total Jobs','Thời vụ/Hợp đồng ngắn hạn','Mới tốt nghiệp', 'Senior', 'Cấp quản lý điều hành'], index=[0])
df.rename(columns={3:'Intern', 0:'Junior', 2: 'Senior', 1: 'Manager/Director'}, inplace = True)


In [61]:
df = df.rename(columns={"Thời vụ/Hợp đồng ngắn hạn": "Intern", "Mới tốt nghiệp": "Junior", 'Cấp quản lý điều hành': 'Manager/Director'})

Vietnamwork_result_sen = df
Vietnamwork_result_sen


Total Jobs  Intern  Junior Senior  Manager/Director
0       12975      20     166    NaN               712

In [ ]:
from datetime import date
Vietnamwork_result_ind.to_csv(f'vietnamwork-ind-{date.today()}.csv')
Vietnamwork_result_sen.to_csv(f'vietnamwork-sen-{date.today()}.csv')